In [ ]:
import numpy as np 
import pandas as pd 
from keras.models import Sequential
from keras.layers import Dense, Dropout
import gc

In [ ]:
!nvidia-smi

In [ ]:
data = pd.read_csv("../input/tabular-playground-series-jun-2022/data.csv")
data = data.drop(columns=["row_id"])

In [ ]:
data.head()

In [ ]:
#We can see that the values are acceptably small and don't have to be normalised
for column in data:
    print(f"min: {min(data[column])}")
    print(f"max: {max(data[column])}")

In [ ]:
model = Sequential()
model.add(Dense(256, activation="relu", input_shape=(79,)))

model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))    # Dropout To Avoid Overfit

model.add(Dense(64, activation="relu"))
model.add(Dropout(0.25))

model.add(Dense(64, activation="relu"))

model.add(Dense(32, activation="relu"))

model.add(Dense(1, activation="linear"))

In [ ]:
for column in data:
    X = data[data[column].notna()]
    #print(f"X initial, shape: {X.shape}")
    #print(X.head())
    
    #becuase -1000 is much lower than any value that naturally occours in the data it'll tell
    #the neural network that this value is missing
    X = X.fillna(-1000)
    #print(f"X after filling nan with -1000, shape: {X.shape}")
    #print(X.head())
    
    Y = X[column]
    print(f"Y std dev: {Y.std()}, Y mean: {Y.mean()}")
    #print(f"Y initail, shape: {Y.shape}")
    #print(Y.head())
    
    X = X.drop(columns=[column])
    #print(f"X after dropping the label, shape: {X.shape}")
    #print(X.head())
    
    model.compile(loss='mean_squared_error', optimizer='adam')
    
    # check if the model doesn't overfit first then for the final submission don't use validations
    model.fit(X, Y, epochs=3, batch_size=256, validation_split = 0.1)
    
    #print("model predictions")
    #print(model.predict(X[0:5]))
    predictions = model.predict(X[0:10])
    print(len(predictions))
    for n in range(10):
        print(f"model predicted: {predictions[n]}, the actual value was: {Y[n]}")
    
    gc.collect()